August 2025

## Trying a model where we first optimize courses, and then optimize times

In [1]:
from docplex.mp.model import Model
import pandas as pd
from pandas import DataFrame
import datetime
#import xlrd
import matplotlib.pyplot as plt 
import numpy as np
import re
import itertools
import math
from datetime import datetime
#loading packages that are needed
# TPS 
from docplex.mp.conflict_refiner import ConflictRefiner
from docplex.mp.relaxer import Relaxer
#This isn't where the infeasibility notebook imports priority from, which is vaguely concerning
from docplex.mp.priority import Priority

In [2]:
# file names
# Using Overlapping_times_version_4.ipynb that automatically renames the output file in a particular way
# True/False flag to rename the overlaps for you
time_option_overlaps_auto = True 
prev_schedule = False # true/false flag to see if needing to run previous schedule stuff
qualtrics_file_short = 'Spring2026_withP_update_preps' #'Math_347_example_Oct_2024_infeasible' #"Larger_example_07-11-2023_with_preps" 
qualtrics_file_name = f'{qualtrics_file_short}.xlsx'
targets_file_name = "Spring_2026_to_merge_with_Qualtrics_with_Cannot_teach_update_credits.xlsx"
# Not needed here because only optimizing prof-course pairs, not times
# time_groups_file_name = 'Meeting_Patterns_July_2025.xlsx'#'Meeting_Patterns_small_Oct_2024.xlsx' #'Meeting_Patterns_Medium_Nov_2024_3.xlsx'
# time_option_file_name = ''
# if time_option_overlaps_auto:
#     time_option_file_name = time_groups_file_name.removesuffix('.xlsx') + '_overlaps.xlsx'

use_course_course_prof = False # if False, ignores preferences and constraints involving similar courses
                               # if True, need to read in the course_course_prof constraints and prefs, 
                               # which are currently in the time groups file (but maybe should be somewhere else)

# Theodore 21 Feb 2025
# Moved this to the bottom of the notebook, 7 Mar 2025
##output_file_name = f'results_{class_file_short}.csv'
# now = datetime.now()
# today_str = now.strftime("%Y:%m:%d_%H:%M:%S")
# output_file_name = f'results_{class_file_short}_{today_str}.csv'


if prev_schedule:
    partial_file =  f'{class_file_short}_results_partial.csv' #partial results to use when schedule is premade
    output_file_name = f'{class_file_short}_results_partial.csv' # Where the resulting schedule will be stored

In [3]:
# constants for the model
max_number_sections_to_be_at_diff_times = 3 # if a course has this number or less sections, they must happen at different times, as in adding a constraint for these courses to not overlap irregardless of professor
pref_scaler = 2.5

## Parse files

In [4]:
# converting files/sheets into df
class_xls_file = pd.ExcelFile(qualtrics_file_name)
#time_xls_file = pd.ExcelFile(time_option_file_name)
#time_group_xls_file = pd.ExcelFile(time_groups_file_name)
targets_xls_file = pd.ExcelFile(targets_file_name)
parseable_sheets = class_xls_file.sheet_names + targets_xls_file.sheet_names #time_xls_file.sheet_names + time_group_xls_file.sheet_names
did_not_parse = []
if prev_schedule:
    partial_results_file = pd.read_csv(partial_file)
    partial_results_file["all_times"]=partial_results_file["all_times"].convert_dtypes(int)
df_preferences = class_xls_file.parse('Preferences')
df_goals = targets_xls_file.parse('Goal_nums')
df_credits = targets_xls_file.parse('class_credits')
#df_times = time_xls_file.parse()
#df_time_groups = time_group_xls_file.parse('Time_Group_Info')
#df_time_pref = class_xls_file.parse("Time_preferences")
df_credits_per_prof = targets_xls_file.parse("Credits_per_prof")
df_back_to_back_pref = class_xls_file.parse("Back_to_back")
df_preps_pref = class_xls_file.parse("Preps") #adding df of preps preferences

if use_course_course_prof:
    if 'Course_course_prof_prefs' in parseable_sheets:
        df_course_course_prof_prefs = time_group_xls_file.parse('Course_course_prof_prefs')
    else:
        did_not_parse.append('Course_course_prof_prefs')
    if 'Course_course_prof_constraints' in parseable_sheets:
        df_course_course_prof_constraints = time_group_xls_file.parse('Course_course_prof_constraints')
    else:
        did_not_parse.append('Course_course_prof_constraints')
# # if 'Course_course_time_prefs' in parseable_sheets:
# #     df_course_course_time_prefs = time_group_xls_file.parse('Course_course_time_prefs')
# # else:
# #     did_not_parse.append('Course_course_time_prefs')
# if 'Course_course_time_constraints'  in parseable_sheets:   
#     df_course_course_time_constraints = time_group_xls_file.parse('Course_course_time_constraints')
# else:
#     did_not_parse.append('Course_course_time_constraints')
# if 'Course_time_prefs' in parseable_sheets:
#     df_course_time_prefs = time_group_xls_file.parse('Course_time_prefs')
# else:
#     did_not_parse.append('Course_time_prefs')
# if 'Course_time_constraints' in parseable_sheets:
#     df_course_time_constraints = time_group_xls_file.parse('Course_time_constraints')
# else:
#     did_not_parse.append('Course_time_constraints')
if 'Cannot_teach' in parseable_sheets:
    df_prof_cannot_teach = targets_xls_file.parse("Cannot_teach") # TPS
else:
    did_not_parse.append('Cannot_teach')
print(f'{did_not_parse=}')

did_not_parse=[]


## Tell me which classes (if any) are missing credit numbers

From among courses with Goal_max > 0 

In [5]:
credit_check = pd.merge(df_goals, df_credits, how="left", left_on="Course", right_on="course_name")
credit_check = credit_check[credit_check["Goal_number_max"] > 0]
credit_check = credit_check[credit_check["number_of_credits"].isna()]
if credit_check.shape[0] > 0:
    raise ValueError("class_credits sheet does not include the following courses:\n" + credit_check["Course"].to_string())
#df_credits

## Check for missing credit numbers for faculty
### and for faculty with missing preferences, among faculty with max_credits > 0

In [6]:
df_credits_per_prof = df_credits_per_prof.loc[df_credits_per_prof["num_credits_max"] > 0,:]
prof_check = pd.merge(df_preferences, df_credits_per_prof, how="outer", left_on="Instructor", right_on="prof_name")
missing_prefs = prof_check.loc[prof_check["Instructor"].isna(), "prof_name"]
missing_prof_credits = prof_check.loc[prof_check["prof_name"].isna(), "Instructor"]
if missing_prefs.shape[0] > 0:
    raise ValueError("No preferences available for the following instructors:\n" + missing_prefs.to_string())
if missing_prof_credits.shape[0] > 0:
    raise ValueError("No min/max credit numbers for the following instructors:\n" + missing_prof_credits.to_string())


### Check spelling of professors' names in Preps
Normally not an issue, because Parse_Qualtrics sets the names to match. But if I forgot to fix a spelling issue in the original download from Qualtrics and then try to change it after running Parse_Qualtrics, this code cell will alert me if I missed one of the sheets.

In [7]:
prof_preps_check = pd.merge(df_credits_per_prof, df_preps_pref, how="outer", left_on="prof_name", right_on="Prof_name")
missing_preps_prefs = prof_preps_check.loc[prof_preps_check["Prof_name"].isna(), "prof_name"]
extra_preps_prof = prof_preps_check.loc[prof_preps_check["prof_name"].isna(), "Prof_name"]
if missing_preps_prefs.shape[0] > 0:
    raise ValueError("No preferences about number of preps available for the following instructors:\n" + missing_preps_prefs.to_string())
if extra_preps_prof.shape[0] > 0:
    raise ValueError("The following instructors have preferences about number of preps but no min/max credit numbers:\n" + extra_preps_prof.to_string())

## formatting the dfs to work well with each other and the model

In [8]:
df_credits_per_prof = df_credits_per_prof.loc[df_credits_per_prof['num_credits_max'] > 0]
df_credits_per_prof.rename(columns = {'prof_name' : 'all_profs'}, inplace=True)
profs_with_enough_creds = df_credits_per_prof['all_profs'].tolist()
#df_credits_per_prof

In [9]:
df_goals.rename(columns = {'Course' : 'all_courses'}, inplace=True)
df_goals = df_goals.loc[df_goals['Goal_number_max'] > 0]
courses_with_enough_sections = df_goals['all_courses'].tolist()
#df_goals

In [10]:
df_preferences.rename(columns = {'Instructor' : 'all_profs'}, inplace=True)
courses_with_enough_sections_and_other_headers = courses_with_enough_sections.append("all_profs")
df_preferences = df_preferences.loc[df_preferences['all_profs'].isin(profs_with_enough_creds),df_preferences.columns.isin(courses_with_enough_sections)].copy()
#df_preferences

In [11]:
df_credits.rename(columns = {'course_name' : 'all_courses'}, inplace=True)
df_credits = df_credits.loc[df_credits['all_courses'].isin(courses_with_enough_sections)].copy()
#df_credits

In [12]:
if use_course_course_prof:
    df_course_course_prof_prefs.drop(columns= 'Notes', inplace = True)
    df_course_course_prof_prefs = df_course_course_prof_prefs.loc[df_course_course_prof_prefs['Course_1'].isin(courses_with_enough_sections)]
    df_course_course_prof_prefs = df_course_course_prof_prefs.loc[df_course_course_prof_prefs['Course_2'].isin(courses_with_enough_sections)]
    # df_course_course_prof_prefs

    df_course_course_prof_constraints.drop(columns= 'Notes', inplace = True)
    df_course_course_prof_constraints = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Course_1'].isin(courses_with_enough_sections)]
    df_course_course_prof_constraints = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Course_2'].isin(courses_with_enough_sections)]
    # df_course_course_prof_constraints

In [13]:
df_prof_cannot_teach.drop(columns='Notes', inplace=True)
df_prof_cannot_teach.rename(columns = {'Prof':'all_profs'}, inplace=True)
df_prof_cannot_teach.rename(columns = {'Course':'all_courses'}, inplace=True)
# df_prof_cannot_teach

In [14]:
## for df of preferences for preps
df_preps_pref.rename(columns = {"Prof_name":"all_profs"}, inplace = True)
df_preps_pref = df_preps_pref.loc[df_preps_pref['all_profs'].isin(profs_with_enough_creds)]
# df_preps_pref

In [15]:
# multiplying the preps preferences by a scalar

colms = ['One','Two','Three','Four']

i = 0
for col in colms:
    df_preps_pref[col] = df_preps_pref[col]*pref_scaler
    i += 1
    
#df_preps_pref

In [16]:
all_profs = df_preferences["all_profs"]
#saving a variable with just the professor names
all_courses = df_goals["all_courses"]
#saving a variable with just the course names


# Feasiblity

In [17]:
min_credits_profs_can_teach = 0 
max_credits_profs_can_teach = 0
min_goal_credits = 0
max_goal_credits = 0

for idx in range(df_credits_per_prof.shape[0]):
    row = df_credits_per_prof.iloc[idx]
    #finds the number of credits a professor can teach at minimum
    min_credits_profs_can_teach = min_credits_profs_can_teach + int(row['num_credits_min'])

#finds min sections or min total credits offered per class
for idx in range(df_goals.shape[0]):
    goal_row = df_goals.iloc[idx]
    #iterates through the credit goals for each class and saves the value for each class section in credit_per_class
    credit_per_class = df_credits.loc[df_credits['all_courses'] == goal_row['all_courses']]
        
    #uncomment the following to see the math that is used to find the minimum credits of a class that should be offered in a semester. 
    # the math should be the minimum preffered sections times the number of credits of the class. 
    # print(credit_per_class)
    # print(goal_row)
    
    for idx_2 in range(credit_per_class.shape[0]):
        credit_row = credit_per_class.iloc[idx_2]
        min_goal_credits = min_goal_credits + (int(goal_row['Goal_number_min']) * int(credit_row["number_of_credits"]))
    # print(credit_row.all_courses + " :\t" + str(min_goal_credits)+ " minimum credits") 
        #uncomment to see the total credits of a class

#finds every teachers maximum credit load saved as max_credits_prof_can_teach
for idx in range(df_credits_per_prof.shape[0]):
    row = df_credits_per_prof.iloc[idx]
    max_credits_profs_can_teach = max_credits_profs_can_teach + int(row['num_credits_max'])
# print(row) # uncomment to see a sample of the rows in df_credits per prof

#finds the maximum sections and credit count for each class and stores it in max_goal_credits
for idx in range(df_goals.shape[0]): 
    goal_row = df_goals.iloc[idx]
    credit_per_class = df_credits.loc[df_credits['all_courses'] == goal_row['all_courses']] #matches the course name found in the credit table with the course name found in the section offering goal table
    for idx_2 in range(credit_per_class.shape[0]):#loops through the table of classes and their standard credit count
        credit_row = credit_per_class.iloc[idx_2]
        max_goal_credits = max_goal_credits + int(goal_row['Goal_number_max'])*int(credit_row['number_of_credits']) # multiplies credits by maximum section offering count
#print(credit_row) # uncomment to see a sample of a row in credit_per_class
#print(goal_row) # uncomment to see a sample of a row in df_goals
        
#the following code checks the range for total available professors credits to the total needed credit offerings 
#print("is " + str(max_credits_profs_can_teach) +" < "+ str(min_goal_credits) + "? If, so, problem.") #uncomment to see comparisons
#print("is " + str(min_credits_profs_can_teach) +" > "+ str(max_goal_credits) + "? If, so, problem.") #uncomment to see second comparison

if max_credits_profs_can_teach < min_goal_credits:
        #print("infeasible")
        raise ValueError(f"Infeasible model: The minimum credits of classes desired is {min_goal_credits}; this is greater than the maximum the professors can teach ({max_credits_profs_can_teach}).")
elif min_credits_profs_can_teach > max_goal_credits:
        #print("infeasible")
        raise ValueError(f"Infeasible model: The maximum credits of classes desired is {max_goal_credits}; this is less than the minimum the professors can teach ({min_credits_profs_can_teach}).")
else:
    print("No obvious infeasibilities, proceed with model")
        
#should also add an error if the number of time options are not enough for the number of credits

No obvious infeasibilities, proceed with model


# Create model and main variables

In [18]:
mdl = Model(name="classes") #creates cplex model

Remove Cannot_teach from the set of combos for which we will create assignment variables.

In [19]:
profs_and_courses = all_courses.to_frame().merge(all_profs, how='cross')
profs_and_courses.rename(columns = {'Class':'all_courses'}, inplace=True)
df_prof_cannot_teach['remove'] = True
profs_and_courses = profs_and_courses.merge(df_prof_cannot_teach, how='left')
profs_and_courses = profs_and_courses[profs_and_courses['remove'].isnull()]
profs_and_courses.drop('remove', axis=1, inplace=True)

In [20]:
profs_and_courses = profs_and_courses.assign(combo_name = profs_and_courses.all_courses + '_with_prof_' + profs_and_courses.all_profs)
assigned = mdl.integer_var_list(profs_and_courses.combo_name) # unlike in main scheduling code, these variables are integers, not binary, because a person could teach > 1 section of a course
#assigned

In [21]:
df_assigned = profs_and_courses
df_assigned['assigned'] = assigned
df_assigned = df_assigned.drop('combo_name', axis=1)

In [22]:
mdl.print_information()

Model: classes
 - number of variables: 851
   - binary=0, integer=851, continuous=0
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Constrain assignment variables to be non-negative

In [23]:
for index, row in df_assigned.iterrows():
    #print(row['assigned'])
    prof = row['all_profs']
    clas = row['all_courses']
    mdl.add_constraint(row['assigned'] >= 0, ctname = f'{prof} cannot teach negative values of {clas}').priority = Priority.MANDATORY

In [24]:
mdl.print_information() # number of constraints = number of variables

Model: classes
 - number of variables: 851
   - binary=0, integer=851, continuous=0
 - number of constraints: 851
   - linear=851
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Constraints from previous schedules would go here

## Constrain each professor to teach the correct number of credits

In [25]:
df_assigned = df_assigned.merge(df_credits, how='left')
for prof, course_assignments in df_assigned.groupby(by='all_profs'):
    mdl.add_constraint(mdl.sum(course_assignments['assigned']*course_assignments['number_of_credits']) <= (df_credits_per_prof.loc[lambda df:df['all_profs'] == prof, 'num_credits_max'].values[0]), ctname = f"max credits per {prof}").priority = Priority.MEDIUM 
    mdl.add_constraint(mdl.sum(course_assignments['assigned']*course_assignments['number_of_credits']) >= (df_credits_per_prof.loc[lambda df:df['all_profs'] == prof, 'num_credits_min'].values[0]), ctname = f"min credits per {prof}").priority = Priority.MEDIUM

In [26]:
mdl.print_information() # This adds a number of constraints = 2*number of instructors

Model: classes
 - number of variables: 851
   - binary=0, integer=851, continuous=0
 - number of constraints: 899
   - linear=899
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Constraining each class to be taught the correct number of times

If you get "Error: Adding trivially infeasible linear constraint: '0 >= 1'", double-check that all courses that should be in Meeting_Patterns_large.xlsx, are there. Don't think we need to re-run Overlapping_times_version_1.ipynb unless the class has a new "type" of meeting pattern.

In [27]:
#Skip this while debugging
#summing all of a class assigment variables and saying that it has to be less than and greater than the min and max number of sections per that class
for class_name in all_courses:
    #print(class_name)
    #print(df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_min"].iloc[0])
    mdl.add_constraint(mdl.sum(df_assigned.loc[lambda df: df["all_courses"] == class_name, "assigned"]) <= (df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_max"].iloc[0]), ctname = f"max number of {class_name} sections").priority = Priority.MEDIUM
    mdl.add_constraint(mdl.sum(df_assigned.loc[lambda df: df["all_courses"] == class_name, "assigned"]) >= (df_goals.loc[lambda df: df["all_courses"] == class_name, "Goal_number_min"].iloc[0]), ctname = f"min number of {class_name} sections").priority = Priority.MEDIUM

In [28]:
mdl.print_information() # This adds a number of constraints = 2 * the number of distinct courses

Model: classes
 - number of variables: 851
   - binary=0, integer=851, continuous=0
 - number of constraints: 993
   - linear=993
 - parameters: defaults
 - objective: none
 - problem type is: MILP


## Constraint: Course-Course Prof Constraints
 - TPS 11/19/24
some courses can't be/have to be taught by the same prof

Avoid = If a prof teaches a section of course_1, they cannot teach any of course_2, and vice versa

Require = If a prof teaches x sections of course_1, they must teach exactly x sections of course_2, and vice versa. This assumes that the same number of sections of course_1 and course_2 are running

You (Theo) should make sure to test if this still works when there are more than one section of course_1 and course_2

In [29]:
# Have not tested this for Courses_only code
if use_course_course_prof:
    #I want to write a feasibility check for the sake of my sanity so I need them before that cell
    #TS 11/18/24
    df_course_course_prof_cts_avoid = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Type'] == "Avoid"]
    df_course_course_prof_cts_require = df_course_course_prof_constraints.loc[df_course_course_prof_constraints['Type'] == "Require"]

    df_course_1_assigned_avoid = df_course_course_prof_cts_avoid.copy().rename(columns = {"Course_1":"all_courses"}).merge(df_assigned)
    df_course_2_assigned_avoid = df_course_course_prof_cts_avoid.copy().rename(columns = {"Course_2":"all_courses"}).merge(df_assigned)

    df_course_1_assigned_require = df_course_course_prof_cts_require.copy().rename(columns = {"Course_1":"all_courses"}).merge(df_assigned)
    df_course_2_assigned_require = df_course_course_prof_cts_require.copy().rename(columns = {"Course_2":"all_courses"}).merge(df_assigned)

    #Following code not in shared drive

    df_course_course_prof_cts_require = df_course_course_prof_cts_require.copy().drop(["Type"], axis = 1)
    # df_course_course_prof_cts_require
    df_goals_class_index = df_goals.set_index(df_goals['all_courses'])
    # df_course_course_prof_cts_require.at[1,'Course_2'] = 'Math 246'
    # print(df_course_course_prof_cts_require)
    for index, row in df_course_course_prof_cts_require.iterrows():
        goals_slice = df_goals_class_index.loc[[row.iloc[0], row.iloc[1]]]
        #print(goals_slice)
        if (goals_slice.iloc[0]["Goal_number_min"] > goals_slice.iloc[1]["Goal_number_max"]) or (goals_slice.iloc[1]["Goal_number_min"] > goals_slice.iloc[0]["Goal_number_max"]):
            raise ValueError(f'you need to run the same number of sections if you are requiring matched profs')
    #     print(row.iloc[1])

In [30]:
# Have not tested this for Courses_only code
if use_course_course_prof:
    if not df_course_1_assigned_avoid.empty:
        for prof in all_profs:
            #               if the sum of assignment variables for prof with course_1 is >= 1,
            #                       then the sum of assignment variables for prof with course_2 <= 0
            mdl.add_if_then(sum(df_course_1_assigned_avoid.loc[df_course_1_assigned_avoid['all_profs']==prof]['assigned'])>= 1, sum(df_course_2_assigned_avoid.loc[df_course_2_assigned_avoid['all_profs']==prof]['assigned']) <= 0).priority = Priority.LOW
            mdl.add_if_then(sum(df_course_2_assigned_avoid.loc[df_course_2_assigned_avoid['all_profs']==prof]['assigned'])>= 1, sum(df_course_1_assigned_avoid.loc[df_course_1_assigned_avoid['all_profs']==prof]['assigned']) <= 0).priority = Priority.LOW
    if not df_course_1_assigned_require.empty:
        for prof in all_profs:
            mdl.add_constraint(sum(df_course_1_assigned_require.loc[df_course_1_assigned_require['all_profs']==prof]['assigned']) == sum(df_course_2_assigned_require.loc[df_course_2_assigned_require['all_profs']==prof]['assigned']), ctname = f'required_to_teach_both_courses_for_{prof}').priority = Priority.HIGH
    

# Preferences about courses

In [31]:
df_class_pref_long = df_preferences.melt(id_vars = ["all_profs"], var_name = "all_courses", value_name = "Preference_value")
df_class_pref_long = df_class_pref_long.rename(columns = {"prof_name":"all_profs"})
#df_time_pref_long = df_time_pref.melt(id_vars = ["all_profs"], var_name = "all_times", value_name = "Preference_value_of_time")
#df_time_pref_long = df_time_pref_long.rename(columns = {"Prof_name":"all_profs"})
df_assigned_with_prefs = df_assigned.merge(df_class_pref_long, on = ["all_profs", "all_courses"])
#df_assigned_with_prefs = df_assigned_with_prefs.merge(df_time_pref_long, on = ['all_profs', 'all_times'])
class_prefs_satisfaction_series = df_assigned_with_prefs["assigned"] * df_assigned_with_prefs['Preference_value']
#time_prefs_satisfaction_series = df_assigned_with_prefs["assigned"] * df_assigned_with_prefs['Preference_value_of_time']
# df_assigned_with_prefs
# summing the preferences across prof class and times

# Number of Preps per Professor (preferences)

In [32]:
#creating binary variable matrix where 0 means professor teaches no sections of class and 
#1 means professor teaches at least 1 section
assigned_any_sections = mdl.binary_var_list(profs_and_courses.combo_name + "_any")

In [33]:
mdl.print_information() # as expected, this doubles the number of variables

Model: classes
 - number of variables: 1702
   - binary=851, integer=851, continuous=0
 - number of constraints: 993
   - linear=993
 - parameters: defaults
 - objective: none
 - problem type is: MILP


Constrain the binary variables to equal 1 iff the integer variables are > 0.

In [34]:
df_assigned['assigned_any'] = assigned_any_sections
#df_assigned
for index, row in df_assigned.iterrows():
    mdl.add_if_then(row['assigned'] >= 1, row['assigned_any'] == 1)
    mdl.add_if_then(row['assigned'] == 0, row['assigned_any'] == 0)

In [35]:
mdl.print_information() # each if_then adds 1 variable and 1 constraint

Model: classes
 - number of variables: 3404
   - binary=2553, integer=851, continuous=0
 - number of constraints: 4397
   - linear=993, indicator=1702, equiv=1702
 - parameters: defaults
 - objective: none
 - problem type is: MILP


Create variables for number of preps

In [36]:
#creating binary variable matrix for number of preps for a professor
#0 being they don't have that many preps and 1 being they have that many preps
#only should have one 1 in each row
num_preps = ["One","Two","Three","Four"]
assigned_num_prep = mdl.binary_var_matrix(keys1 = all_profs, keys2 = num_preps, name = "%s_with_%s")
df_assigned_num_prep = DataFrame({'assigned_num_prep':assigned_num_prep})
df_assigned_num_prep.index.names = ['all_profs', 'num_preps']
df_assigned_num_prep = df_assigned_num_prep.reset_index()
#df_assigned_num_prep

,all_profs,num_preps,assigned_num_prep
0,AbraBrisbin,One,AbraBrisbin_with_One
1,AbraBrisbin,Two,AbraBrisbin_with_Two
2,AbraBrisbin,Three,AbraBrisbin_with_Three
3,AbraBrisbin,Four,AbraBrisbin_with_Four
4,JulianAntolinCamarena,One,JulianAntolinCamarena_with_One
...,...,...,...
91,RyanAndJennyHarrison,Four,RyanAndJennyHarrison_with_Four
92,KatieElliott,One,KatieElliott_with_One
93,KatieElliott,Two,KatieElliott_with_Two
94,KatieElliott,Three,KatieElliott_with_Three


In [41]:
mdl.print_information()

Model: classes
 - number of variables: 3596
   - binary=2745, integer=851, continuous=0
 - number of constraints: 4613
   - linear=1017, indicator=1798, equiv=1798
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [39]:
#summing up all class_assigned variables and setting assigned_num_prep to 1 based on how many preps
#preps are summation of all class_assigned variables for a professor
for prof in all_profs:
    df_class_assigned_prof = df_assigned.loc[df_assigned['all_profs']==prof]
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['assigned_any']) == 1, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='One']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['assigned_any']) == 2, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='Two']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['assigned_any']) == 3, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=='Three']['assigned_num_prep'].unique()[0] == 1)
    mdl.add_if_then(mdl.sum(df_class_assigned_prof['assigned_any']) >= 4, df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof].loc[df_assigned_num_prep['num_preps']=="Four"]['assigned_num_prep'].unique()[0] == 1)
    mdl.add_constraint(mdl.sum(df_assigned_num_prep.loc[df_assigned_num_prep['all_profs']==prof]['assigned_num_prep'])<=1)

In [40]:
mdl.print_information()

Model: classes
 - number of variables: 3596
   - binary=2745, integer=851, continuous=0
 - number of constraints: 4613
   - linear=1017, indicator=1798, equiv=1798
 - parameters: defaults
 - objective: none
 - problem type is: MILP


In [42]:
#adding prefs to rotated df_preps_pref dataframe
df_preps_pref_melt = pd.melt(df_preps_pref, id_vars=['all_profs'], value_vars=['One', 'Two', 'Three', 'Four'], var_name = 'num_preps', value_name = 'prefs')
df_assigned_num_prep = df_assigned_num_prep.merge(df_preps_pref_melt, on = ['all_profs', 'num_preps'])

In [43]:
preps_prefs_satisfaction_series = df_assigned_num_prep['assigned_num_prep']*df_assigned_num_prep['prefs']

# Similar Courses Preferences

In [45]:
# Have not tested in Courses_only
if use_course_course_prof:
    df_course_course_similar = pd.DataFrame(columns = ['Course_1_Course_2','course_course_assigned'],index=range(df_course_course_prof_prefs.shape[0]))

    # ind is the index of the dataframe and it iterates down the rows, getting the two course preferences
    for ind in range(df_course_course_prof_prefs.shape[0]):
        df_course_course_similar_1 = df_class_assigned[df_class_assigned['all_courses'] == df_course_course_prof_prefs['Course_1'].iloc[ind]]
        df_course_course_similar_2 = df_class_assigned[df_class_assigned['all_courses'] == df_course_course_prof_prefs['Course_2'].iloc[ind]]
        df_course_course_similar.loc[ind,'Course_1_Course_2'] = "%s_%s" % (df_course_course_prof_prefs['Course_1'].iloc[ind],df_course_course_prof_prefs['Course_2'].iloc[ind])
        df_mults = df_course_course_similar_1.copy()
        df_mults = df_course_course_similar_1['class_assigned'].values*df_course_course_similar_2['class_assigned'].values
        df_course_course_similar.loc[ind,'course_course_assigned'] = df_course_course_prof_prefs['Pref'].iloc[ind]*sum(df_mults)
        course_course_prof_prefs_satisfaction_series = df_course_course_similar['course_course_assigned']
else:
    course_course_prof_prefs_satisfaction_series = pd.Series([])

# adding the objective to the model

In [47]:
satisfaction_elements = pd.concat([class_prefs_satisfaction_series,
                                   #time_prefs_satisfaction_series, 
                                   preps_prefs_satisfaction_series, 
                                   ##pd.Series(course_course_time_pref_series), 
                                   #pd.Series(course_time_pref_series),
                                   course_course_prof_prefs_satisfaction_series, 
                                   #pd.Series(df_time_assigned_double['b2b_pref_list_v2'])
                                  ]) #Only the last element of this list should have changed

total_satisfaction = mdl.sum(satisfaction_elements) 
mdl.maximize(total_satisfaction)

mdl.print_information()

Model: classes
 - number of variables: 3596
   - binary=2745, integer=851, continuous=0
 - number of constraints: 4613
   - linear=1017, indicator=1798, equiv=1798
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


# Solve model

In [48]:
solution = mdl.solve() #clean_before_solve = True, will restart the model, whereas false (default) will pick up where last model left off
mdl.report()

* model classes solved with objective = 648.000


In [49]:
solve_status = mdl.get_solve_status()
print(solve_status)
print(mdl.solve_details.time) # is this in seconds? Seems like it.

if solve_status.name == 'INFEASIBLE_SOLUTION':
    infeasible = True
else:
    infeasible = False

JobSolveStatus.OPTIMAL_SOLUTION
203.7958791255951


# Relax Solution, if necessary
TPS 11/19/24

In [50]:
if infeasible:

    cr = ConflictRefiner()
    #type(cr)
    conflicting_constraints = cr.refine_conflict(mdl, display = True)
    conflicting_constraints.display()

In [51]:
if infeasible:

    rx = Relaxer()
    #type(rx)
    relaxed_solution = rx.relax(mdl)
    rx.print_information()
    relaxed_solution.display()

In [52]:
if infeasible:
    df_solution = relaxed_solution.as_df()
else:
    df_solution = solution.as_df()
df_solution

,name,value
0,Math 10_with_prof_KatrinaRothrock,1.0
1,Math 20_with_prof_ChloeLewis,2.0
2,Math 20_with_prof_KatrinaRothrock,1.0
3,Math 50_with_prof_ChristopherDavis,4.0
4,Math 104_with_prof_aBaMbirika,2.0
...,...,...
998,_bool#1780,1.0
999,_bool#1784,1.0
1000,_bool#1788,1.0
1001,_bool#1794,1.0


In [53]:
# Drop placeholder "_bool" variables
solution=df_solution.astype(str)
solution=solution.iloc[[True if (type(solution.iloc[a]['name']) is str and not solution.iloc[a]['name'].startswith("_bool")) else False for a in range(solution.shape[0])]]
solution

,name,value
0,Math 10_with_prof_KatrinaRothrock,1.0
1,Math 20_with_prof_ChloeLewis,2.0
2,Math 20_with_prof_KatrinaRothrock,1.0
3,Math 50_with_prof_ChristopherDavis,4.0
4,Math 104_with_prof_aBaMbirika,2.0
...,...,...
974,ChrisHlas_with_Two,1.0
975,ChristopherDavis_with_Two,1.0
976,Marie-ClaireKoissi-K_with_Two,1.0
977,RyanAndJennyHarrison_with_Four,1.0


In [57]:
#solution.iloc[45:55,:]

,name,value
45,Math 460_with_prof_HerschelDay,1.0
46,DS 140_with_prof_JessicaKraker,2.0
47,DS 150_with_prof_AbhinavChand,2.0
48,DS 250_with_prof_AbhinavChand,1.0
49,DS 710_with_prof_JulianAntolinCamarena,1.0
50,DS 740_with_prof_AbraBrisbin,1.0
51,Math 10_with_prof_KatrinaRothrock_any,1.0
52,Math 20_with_prof_ChloeLewis_any,1.0
53,Math 20_with_prof_KatrinaRothrock_any,1.0
54,Math 50_with_prof_ChristopherDavis_any,1.0


In [55]:
# pd.concat([solution, total])
mid_df= solution.merge(df_assigned.astype(str),left_on="name", right_on="assigned")
# new_df=new_df.loc[new_df.value==1]
#mid_df['all_times'] = mid_df['all_times'].astype(int)
#mid_df.to_csv("test.csv", index=False) # for debugging the extra row
mid_df

,name,value,all_courses,all_profs,assigned,number_of_credits,assigned_any
0,Math 10_with_prof_KatrinaRothrock,1.0,Math 10,KatrinaRothrock,Math 10_with_prof_KatrinaRothrock,4,Math 10_with_prof_KatrinaRothrock_any
1,Math 20_with_prof_ChloeLewis,2.0,Math 20,ChloeLewis,Math 20_with_prof_ChloeLewis,4,Math 20_with_prof_ChloeLewis_any
2,Math 20_with_prof_KatrinaRothrock,1.0,Math 20,KatrinaRothrock,Math 20_with_prof_KatrinaRothrock,4,Math 20_with_prof_KatrinaRothrock_any
3,Math 50_with_prof_ChristopherDavis,4.0,Math 50,ChristopherDavis,Math 50_with_prof_ChristopherDavis,1,Math 50_with_prof_ChristopherDavis_any
4,Math 104_with_prof_aBaMbirika,2.0,Math 104,aBaMbirika,Math 104_with_prof_aBaMbirika,4,Math 104_with_prof_aBaMbirika_any
5,Math 106_with_prof_CarolynOtto,1.0,Math 106,CarolynOtto,Math 106_with_prof_CarolynOtto,4,Math 106_with_prof_CarolynOtto_any
6,Math 107_with_prof_ColleenDuffy,1.0,Math 107,ColleenDuffy,Math 107_with_prof_ColleenDuffy,4,Math 107_with_prof_ColleenDuffy_any
7,Math 109_with_prof_AihamHassan,3.0,Math 109,AihamHassan,Math 109_with_prof_AihamHassan,4,Math 109_with_prof_AihamHassan_any
8,Math 109_with_prof_RobertBerg,3.0,Math 109,RobertBerg,Math 109_with_prof_RobertBerg,4,Math 109_with_prof_RobertBerg_any
9,Math 111_with_prof_RobertBerg,1.0,Math 111,RobertBerg,Math 111_with_prof_RobertBerg,4,Math 111_with_prof_RobertBerg_any


In [58]:
# This version is for exporting to csv
df_schedule_to_export=mid_df.filter(["all_profs","all_courses","value", "number_of_credits"])
df_schedule_to_export = df_schedule_to_export.rename(columns={"value":"Number_of_sections"})

In [59]:
now = datetime.now()
today_str = now.strftime("%Y:%m:%d_%H:%M:%S")
output_file_name = f'Courses_only_{qualtrics_file_short}_{today_str}.csv'

df_schedule_to_export.to_csv(output_file_name, index=False)